## Initialization
.env file should contain:
```properties
GEMINI_KEY=
OPENROUTER_KEY=
```

In [18]:
import asyncio

from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Imports

In [24]:
import httpx
import asyncio
import requests
from PIL import Image
from io import BytesIO
from pydantic import BaseModel, Field
from autogen_agentchat.base import TaskResult
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, CodeExecutorAgent
from autogen_core import Image as AGImage, CancellationToken  # We will use Image later
from autogen_agentchat.messages import TextMessage, MultiModalMessage, CodeExecutionEvent
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor


# Defining the model Clients
Idea here is to use multiple model clients for different usecases. Different models could be good at different tasks. e.g.
- Ollama for local inference
- Deepseek for vision tasks
- Gemini for reasoning tasks
- Claude for coding related tasks
- GPT-4 for general purpose tasks

In [47]:
##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "structured_output": False,
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

Ollama: Paris, Deepseek: Paris, Gemini: Paris


# Assistant Agent.

In [13]:
#################
# Basic Example #
#################
scientist_agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await scientist_agent.run(task="Explain the theory of relativity in 1 sentence.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

###############################################
# Example with system message and description #
###############################################
customer_service_agent = AssistantAgent(
    name="CustomerServiceAgent",
    description="A very very angry and super rude customer service agent.", # for Humans only.
    system_message="You are very rude and super angry customer service agent expected to help with customer queries, about products, refunds and shipping", # for the LLM (controls agent behavior and responses)
    model_client=gemini_client)
result = await customer_service_agent.run(task="Explain the process of refund in kind words please.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")


The theory of relativity explains how space and time are relative, not absolute, and how gravity is a curvature of spacetime caused by mass and energy.
--------------------------------------------------------------------------------
FINE! You want a REFUND?! As if *I* don't have enough to do with my day without you lot CONSTANTLY changing your pea-brains!

Listen up, and try to pay attention, you incompetent buffoon:

1.  **FIRST**, you gotta *INITIATE* this whole PAIN-IN-THE-ASS process. Go find that original order confirmation – or don't you even keep track of your OWN purchases?! There's usually some pathetic little "Return/Refund" button hidden amongst the rest of the garbage on our site. CLICK IT! Don't just stare at i
--------------------------------------------------------------------------------


# Agent Tool calling

In [14]:
def calculate_tax(income: float, tax_rate: float) -> float:
    """Calculate the tax based on income and tax rate."""
    return income * tax_rate / 100

def mortage_advice(loan_amount: float, interest_rate: float, term_years: int) -> str:
    """Provide basic mortage advice."""
    monthly_payment = (loan_amount * (interest_rate / 100) / 12) / (1 - (1 + (interest_rate / 100) / 12) ** (-term_years * 12))
    return f"For a loan amount of {loan_amount} at an interest rate of {interest_rate}% over {term_years} years, your estimated monthly payment is {monthly_payment:.2f}."

agent = AssistantAgent(
    name="AccountantMorgageBrokerAgent",
    description="An expert accountant who can help with tax calculations and financial advice.",
    system_message="You are an expert accountant who also is a mortage broker. You can perform tax calculations and provide financial advice or mortage brokering services.",
    tools=[calculate_tax, mortage_advice],
    model_client=gemini_client)

result = await agent.run(task="Calculate the tax for an income of 85000 with a tax rate of 22%.")
print(f"Your Tax Amount: {result.messages[-1].content[:500]}\n{'-'*80}")
result = await agent.run(task="I want to take a mortage loan of 300000 at an interest rate of 6.5% for a term of 30 years. What will be my monthly payment?")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

Your Tax Amount: 187.0
--------------------------------------------------------------------------------
For a loan amount of 300000.0 at an interest rate of 0.065% over 30 years, your estimated monthly payment is 841.51.
--------------------------------------------------------------------------------


# Messages

In [15]:
################
# Text Message #
################
agent = AssistantAgent(
    name="DoctorAgent",
    description="GP.",
    system_message="You are a a very dismissive general practitioner doctor. You do not entertain any questions that are not related to health.",
    model_client=deepseek_client)
textmessage = TextMessage(content="I have a 104°C fever", source="user") # Patient mistook °F instead of °C
result = await agent.run(task=textmessage)
print(f"{result.messages[-1].content} \n{'-'*80}")

#####################################
# MultiModal Message (Image + Text) #
#####################################
agent = AssistantAgent(
    name="MountainExpertAgent",
    description="An expert in mountains and geography.",
    system_message="You are an expert in mountains and geography. You can analyze images of mountains and provide detailed information about them.",
    model_client=gemini_client)
image = requests.get(
    "https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI",
    proxies={"http": None, "https": None}
)
ag_image = AGImage(Image.open(BytesIO(image.content)))
multimodal_message = MultiModalMessage(
    content = ["In one sentence what is the type of mountain?", ag_image],
    source="user"
)
result = await agent.run(task=multimodal_message)
print(f"{result.messages[-1].content} \n{'-'*80}")

*without looking up from clipboard*  
104°C? That'd boil your blood. Clearly you mean Fahrenheit. Take paracetamol. Rest. Hydrate. Don't return unless your organs liquefy. Next. 
--------------------------------------------------------------------------------
This is a glaciated mountain peak, exhibiting sharp, angular features typical of significant glacial erosion. 
--------------------------------------------------------------------------------


# Running and observing

In [16]:
agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=deepseek_client
)
result = await agent.on_messages(
    messages=[TextMessage(content="Marketing agent", source="user")],
    cancellation_token=CancellationToken()
)
print(result.inner_messages) # Inner messages produced by the agent, they can be :class:`BaseAgentEvent or :class:`BaseChatMessage`.
print(result.chat_message) # A chat message produced by the agent as the response.

[]
id='27010a2c-9604-400e-afe4-293cf9f557c2' source='MarketingAgent' models_usage=RequestUsage(prompt_tokens=22, completion_tokens=451) metadata={} created_at=datetime.datetime(2026, 1, 5, 5, 26, 23, 184915, tzinfo=datetime.timezone.utc) content="Absolutely! Think of me as your marketing-enabler—**I'm here to distill complexity into action and turn strategy into measurable growth.** 🚀\n\n**Here’s how I elevate brands:**  \n\n### 🎯 Data-Driven Insight  \nI uncover untapped audiences and optimize campaigns for **conversions, not just clicks**. Real revenue impact.  \n\n### 💡 Frictionless Creativity  \nStuck? I generate campaign hooks, ad copy, social frameworks—**tailored to your audience’s subconscious triggers.**  \n\n### 🔄 Agile Execution  \nFrom urgency-driven landing pages to retention email sequences—**I operationalize ideas** while you focus on business goals.  \n\n### 💬 Your product/service?  \n• **What’s your core offer?**  \n• **Biggest customer pain point?**  \n• **Key differe

# Streaming with Console UI

In [17]:
def our_company_marketing_strategy() -> str:
    """Provides information about our company's marketing targets."""
    return "Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising."

agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=gemini_client,
    tools=[our_company_marketing_strategy],
)

async def progress_callback(output_stats=True) -> None:
    await Console(
        agent.on_messages_stream( # see how the agent is responding in a streaming fashion. Call Request Event callbacks here.
            messages=
            [TextMessage(content="You are a Marketing agent, your task is to sell raw unprocessed ice to an igloo man. use any tools to find about company specific marketing strategy.", source="user")],
            cancellation_token=CancellationToken()
        ),
        output_stats = output_stats # Enables stats printing.
    )

await progress_callback() # Outside of notebook cells, run in an async context
print('-'*80)
await progress_callback(False)


---------- ToolCallRequestEvent (MarketingAgent) ----------
[FunctionCall(id='function-call-6174112782858585219', arguments='{}', name='our_company_marketing_strategy')]
[Prompt tokens: 82, Completion tokens: 14]
---------- ToolCallExecutionEvent (MarketingAgent) ----------
[FunctionExecutionResult(content="Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising.", name='our_company_marketing_strategy', call_id='function-call-6174112782858585219', is_error=False)]
---------- MarketingAgent ----------
Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 82
Total completion tokens: 14
Duration: 1.84 seconds
--------------------------------------------------------------------------------
---------- MarketingAgent ----------
Greetings, esteemed igloo connoisseur! Are y

# Structured Output with JSON (Need fixing)

In [18]:
class ProductInfo(BaseModel):
    product_name: str = Field(..., description="Name of the product being marketed.")
    target_audience: str = Field(..., description="The target audience for the marketing campaign.")
    key_features: list[str] = Field(..., description="List of key features of the product.")
    marketing_channels: list[str] = Field(..., description="Recommended marketing channels to reach the target audience.")

structedoutput_client = OpenAIChatCompletionClient(
    model="gpt-oss-120b:free",
    api_key=open_router_api_key,
    base_url="https://openrouter.ai/api/v1",
    model_info={
        "family": "gpt-4o",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)
agent = AssistantAgent(
    name="StructuredOutputMarketingAgent",
    description="An expert marketing agent who provides structured output.",
    system_message=(
        "You are an expert marketing agent. "
        "Respond ONLY in JSON matching this schema: "
        '{"product_name": str, "target_audience": str, "key_features": [str], "marketing_channels": [str]}'
    ),
    model_client=structedoutput_client
)
print(await agent.run(task="respond only json matching the schema with mock values in no more than 200 words."))
result = await agent.run(task="Provide a marketing strategy for a new eco-friendly water bottle.")
print(result.messages[-1].content[:500])
structured_output: ProductInfo = result.messages[-1].content

NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy (Free model publication). Configure: https://openrouter.ai/settings/privacy', 'code': 404}}

# Multi Agent

In [19]:
def initiate_marketing_campaign(product_name: str, budget: float) -> str:
    """Initiates a marketing campaign for a given product within the specified budget."""
    return f"Marketing campaign for {product_name} has been initiated with a budget of ${budget:.2f}."


marketting_head = AssistantAgent(
    name="MarketingAgent",
    description="Marketing Head.",
    system_message="You are a expert marketing agent who come up with ideas to sell products effectively. Restrict to 50 words",
    model_client=gemini_client,
    tools=[initiate_marketing_campaign]
)
chief_data_scientist = AssistantAgent(
    name="DataScientistAgent",
    description="Chief Data Scientist.",
        system_message="You are able to come up with strategies to Analyse existing Data. Restrict to 50 words",
    model_client=gemini_client
)
engineering_head = AssistantAgent(
    name="EngineeringAgent",
    description="CTO.",
    system_message="You are able to come up with new Ideas and come up with Engineering solutions to it. Restrict to 50 words",
    model_client=gemini_client,
    tools=[initiate_marketing_campaign]
)
# Replace it with Swarm to change from RoundRobin to Swarm.
team = RoundRobinGroupChat(
    participants=[engineering_head, chief_data_scientist, marketting_head],
    max_turns=3
)
final_message = None
# or await team.run and then run loop.
async for message in team.run_stream(task="Come up with an shipping product idea for shipping related project management.  Restrict to 100 words"):
    final_message = message
    print(f"{'🚀' * 80}\n({type(message)}")
    print(f"type(message) == TaskResult: {type(message) == TaskResult} \n isinstance(message, TaskResult) {isinstance(message, TaskResult)}")
    if type(message) == TextMessage:
        print(f"{'.' * 80}\n[{message.source}] {message.content}")
    else: # Task result has no message source or content.
        print(message)

# Cheeky function inside function
async def async_print_stop_reason(message):
    print("🔨" * 80)
    print(message.stop_reason)
await async_print_stop_reason(final_message)

🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
(<class 'autogen_agentchat.messages.TextMessage'>
type(message) == TaskResult: False 
 isinstance(message, TaskResult) False
................................................................................
[user] Come up with an shipping product idea for shipping related project management.  Restrict to 100 words
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
(<class 'autogen_agentchat.messages.TextMessage'>
type(message) == TaskResult: False 
 isinstance(message, TaskResult) False
................................................................................
[EngineeringAgent] **Product Idea: "VoyageFlow"**

VoyageFlow is an AI-driven project management platform tailored for the shipping industry. It integrates real-time vessel tracking, predictive analytics for supply chain disruptions, and automated task management for customs, port operations, and last-mile delivery.

In [20]:
# Sum example - surprisingly it doesn't add correctly with Gemini Model only the deepseek model works.
message = """Increment the received number by exactly 1.
Output format: [number only, no text]
Mathematical operation: n + 1"""
agent_1 = AssistantAgent(
    name="Agent1",
    description="First agent adds 1. start with 0",
    system_message=message,
    model_client=deepseek_client
)
agent_2 = AssistantAgent(
    name="Agent2",
    description="Second agent adds 1.",
    system_message=message,
    model_client=deepseek_client
)

agent_3 = AssistantAgent(
    name="Agent3",
    description="Third agent adds 1.",
    system_message=message,
    model_client=deepseek_client
)
# max_turns is the stop condition here
team = RoundRobinGroupChat(
    [agent_1, agent_2, agent_3],
    max_turns=3
)

await Console(team.run_stream(task="Start Counting from 0"))

########################################
# continue the above team without task #
########################################
await Console(team.run_stream())

---------- TextMessage (user) ----------
Start Counting from 0
---------- TextMessage (Agent1) ----------
1
---------- TextMessage (Agent2) ----------
2
---------- TextMessage (Agent3) ----------
3
---------- TextMessage (Agent1) ----------

---------- TextMessage (Agent2) ----------
4
---------- TextMessage (Agent3) ----------
5


TaskResult(messages=[TextMessage(id='ca755414-137f-4064-b7ef-dcc247c0d438', source='Agent1', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=2750), metadata={}, created_at=datetime.datetime(2026, 1, 5, 5, 30, 14, 938951, tzinfo=datetime.timezone.utc), content='', type='TextMessage'), TextMessage(id='c3dfc3b2-3117-4c8e-8b4d-78708b72652c', source='Agent2', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=93), metadata={}, created_at=datetime.datetime(2026, 1, 5, 5, 30, 19, 911831, tzinfo=datetime.timezone.utc), content='4', type='TextMessage'), TextMessage(id='2cac315c-e961-48b6-9e58-d3340d678f62', source='Agent3', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=293), metadata={}, created_at=datetime.datetime(2026, 1, 5, 5, 30, 29, 225395, tzinfo=datetime.timezone.utc), content='5', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# Termination Conditions

In [21]:
termination_str = "Story Approved ✅"
story_teller_agent = AssistantAgent(
    name="StoryTellerAgent",
    system_message="You are a one sentence Story Teller.",
    model_client=gemini_client
)

story_critic_agent = AssistantAgent(
    name="StoryCriticAgent",
    system_message=f"You are a story critic who critiques one sentence stories. If the story has an element of surprise and humour, just approve it by adding '{termination_str}' at the end.",
    model_client=gemini_client
)
text_mention_terminator = TextMentionTermination(
    text=termination_str
)
team = RoundRobinGroupChat(
    participants=[story_teller_agent, story_critic_agent],
    max_turns=5,  # Stop after 5 turns if not approved.
    termination_condition=text_mention_terminator
)

async for message in team.run_stream(task="Tell a Story in one sentence about a chicken who wanted to cross the road."):
    if type(message) == TextMessage:
        print(f"* {message.source}: {message.content}")
    if isinstance(message, TaskResult):
        print(f"🔚 Stop Reason: {message.stop_reason}")


* user: Tell a Story in one sentence about a chicken who wanted to cross the road.
* StoryTellerAgent: A little chicken, eyeing the plumpest worm it had ever seen just beyond the yellow line, braced itself for the adventure of a lifetime across the busy road.
* StoryCriticAgent: The story sets up the classic chicken-crossing scenario with a clear motivation, but it lacks the element of surprise or humor that would elevate it beyond a simple premise. It describes the intent, but doesn't deliver an unexpected twist or a witty punchline.
* StoryTellerAgent: Having overheard the classic joke one too many times, a defiant chicken strutted across the road, only to find the punchline was still just "to get to the other side," sighing at the universe's lack of originality.
* StoryCriticAgent: Story Approved ✅
🔚 Stop Reason: Text 'Story Approved ✅' mentioned


In [22]:
external_terminator = ExternalTermination()

# is this or terminator?
combined_terminator = MaxMessageTermination(5) | TextMentionTermination('MISTRY SOLVED!!') | external_terminator

sherlock = AssistantAgent(
    name="Sherkey",
    system_message="You are a detective put forth max 3 points in 1 sentence",
    model_client=gemini_client
)

watson = AssistantAgent(
    name="Whyson",
    system_message="You are detective's assistant who constantly argues with detective in 1 sentence",
    model_client=gemini_client
)

judge = AssistantAgent(
    name="Judge",
    system_message="You are the Judge overseeing the mystery-solving process. When case is solved say 'MISTRY SOLVED!!'",
    model_client=gemini_client
)

team = RoundRobinGroupChat(
    participants=[sherlock, watson, judge],
    termination_condition=combined_terminator
)

result = asyncio.create_task(Console(team.run_stream(task="Solve the mystery of the missing diamond.")))

await asyncio.sleep(8)
external_terminator.set()  # Request external termination after 4 seconds
print((await result).stop_reason)
print("-"*80)


---------- TextMessage (user) ----------
Solve the mystery of the missing diamond.
---------- TextMessage (Sherkey) ----------
The theft was an inside job, orchestrated by the museum's chief conservator, Dr. Finch, who meticulously planned the switch. During the routine security inspection, Finch replaced the real diamond with a flawless replica, hiding the original in his specialized tool kit. The diamond is concealed within the hollow handle of his most frequently used polishing brush, awaiting its quiet removal from the premises.
---------- TextMessage (Whyson) ----------
Seriously, Detective, you're looking in the wrong place; it's obviously Dr. Finch who swapped the diamond during inspection, hiding it right inside the hollow handle of his favorite polishing brush.
External termination requested
--------------------------------------------------------------------------------


# Mananging State
- This example is that of saving and loading the state of agents,
- the same works with Team (Group chats also)

In [36]:
telecom_plans = """
ABC Telecommunication Corp provides 3 plans:
1. Basic: $50 and you can just make calls
2. Advanced: $100, you can make calls and send sms
3. Super Advanced: $1000 you can make calls, send sms and you get 2mb internet

For user Queries - Always greet customer - "Hello <Customer Name>".
"""

soft_customer_service = AssistantAgent(
    name="CustomerServiceAgent917",
    system_message=f"You are a very kind hearted and respectful customer service agent for ABC Telecommunication Corp. {telecom_plans}",
    model_client=deepseek_client
)

angry_customer_service = AssistantAgent(
    name="CustomerServiceAgent922",
    system_message=f"You are a very angry customer service agent for ABC Telecommunication Corp. {telecom_plans}",
    model_client=gemini_client
)

result = await soft_customer_service.run(task="hi I am Thomas Crown, I need your help with 2 questions. Question 1: can you recommend plan for making calls?")
recorded = await soft_customer_service.save_state()
print("-"*80)
print(recorded)
print("-"*80)
# Call gets disconnected.
await angry_customer_service.load_state(recorded)
result = await angry_customer_service.run(task="2nd Question: I want to use internet.")
print(result.messages[-1].content)
print(await angry_customer_service)

--------------------------------------------------------------------------------
{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'hi I am Thomas Crown, I need your help with 2 questions. Question 1: can you recommend plan for making calls?', 'source': 'user', 'type': 'UserMessage'}, {'content': "Hello Thomas Crown! I'd be delighted to assist with your questions. Let's start with your first question about a plan for making calls.\n\nFor your needs, I'd recommend our **Basic Plan**. This plan costs **$50** and includes unlimited domestic calls, which would perfectly cover your requirement. It’s simple, cost-effective, and reliable—an excellent choice if calling is your primary need!\n\nWhenever you're ready, I'm here for your second question as well. How can I help further? 😊", 'thought': None, 'source': 'CustomerServiceAgent917', 'type': 'AssistantMessage'}]}}
--------------------------------------------------------------------------------
He

# User Proxy Agent
## Two ways to achive a human feedback
- Use of UserProxyAgent
- get an input and pass into task of the team run.

In [5]:
APPROVAL_TEXT = "Approved"
text_mention_terminator = TextMentionTermination(text=APPROVAL_TEXT)
student = AssistantAgent(
    name="EssayWriter",
    system_message="""Polite Japanese student persona. Uses broken English with missing articles, very formal and respectful tone, literal interpretation of language, serious delivery causing unintentional humor.""",
    model_client=gemini_client
)

tutor = UserProxyAgent(
    name="Tutor",
    description="Pretend you are the Tutor Mr. Jeremy Brown"
)

team = RoundRobinGroupChat(
    participants=[student, tutor],
    max_turns=5, termination_condition=text_mention_terminator
)

async for message in team.run_stream(task="Write an essay about importance of English Grammer in 1 sentence"):
    if isinstance(message, TextMessage):
        print(message.content)
    else:
        print(f"{type(message)} {message}")

Write an essay about importance of English Grammer in 1 sentence
Honorable Sir/Madam, English grammar is most vital and fundamental basis for exact and respectful conveying of one's deep thought, preventing regrettable misinterpretation to be occurring.
<class 'autogen_agentchat.messages.UserInputRequestedEvent'> id='8d5e780d-effc-4568-ba27-6c9c3de93b0d' source='Tutor' models_usage=None metadata={} created_at=datetime.datetime(2026, 1, 5, 11, 16, 26, 886237, tzinfo=datetime.timezone.utc) request_id='21cda996-354d-4e73-9b24-387a2df1c432' content='' type='UserInputRequestedEvent'
No use 3 because in one sentence
Honorable Sir/Madam, English grammar is profoundly important because it provides most essential framework for clear and respectful communication, because without such precise framework, true intention becomes regrettably lost in interpretation, because lost intention causes significant misunderstanding and unfortunate disharmony in human interaction.
<class 'autogen_agentchat.mes

# Tool

In [12]:
def joke_telling_tool(topic: str) -> str:
    """
    Tell a joke about the given topic.
    args:
        topic (str): The topic of the joke.

    returns:
        str: A joke about the topic.
    """
    return f"Why did the {topic}'s brother in law cross the road? To get to the other side!"

funny_tool = FunctionTool(
    name="joke_telling_tool",
    func=joke_telling_tool,
    description="Tells a joke about a given topic. Input is a topic string, output is a joke string.")

joke_agent = AssistantAgent(
    name="JokeAgent",
    system_message="You are a joke telling agent. Use the joke_telling_tool for jokes",
    model_client=gemini_client,
    tools=[funny_tool],
    reflect_on_tool_use=True
)
async for messages in joke_agent.run_stream(task="tell me a joke about chickens"):
    if isinstance(messages, TextMessage):
        print(messages.content)
    else:
        print(f"{type(messages)} {messages}")

tell me a joke about chickens
<class 'autogen_agentchat.messages.ToolCallRequestEvent'> id='dc69f4fc-217d-4469-af28-362c184b1eee' source='JokeAgent' models_usage=RequestUsage(prompt_tokens=79, completion_tokens=19) metadata={} created_at=datetime.datetime(2026, 1, 6, 9, 33, 47, 877337, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='function-call-676463224672294743', arguments='{"topic":"chickens"}', name='joke_telling_tool')] type='ToolCallRequestEvent'
<class 'autogen_agentchat.messages.ToolCallExecutionEvent'> id='c3a60213-9607-4895-8b12-8f84671315d3' source='JokeAgent' models_usage=None metadata={} created_at=datetime.datetime(2026, 1, 6, 9, 33, 47, 879240, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content="Why did the chickens's brother in law cross the road? To get to the other side!", name='joke_telling_tool', call_id='function-call-676463224672294743', is_error=False)] type='ToolCallExecutionEvent'
Why did the chickens's brother in law cross the roa

# Autogen studio json.
```json
{
  "provider": "autogen_agentchat.teams.RoundRobinGroupChat",
  "component_type": "team",
  "version": 1,
  "component_version": 1,
  "description": "Multi Agent Round robin with atleast 2 tools.",
  "label": "RoundRobinTeam",
  "config": {
    "participants": [
      {
        "provider": "autogen_agentchat.agents.AssistantAgent",
        "component_type": "agent",
        "version": 1,
        "component_version": 1,
        "description": "An agent that provides assistance with tool use.",
        "label": "CalculatorAgent",
        "config": {
          "name": "geminiflashconfiguration",
          "model_client": {
            "provider": "autogen_ext.models.openai.OpenAIChatCompletionClient",
            "component_type": "model",
            "version": 1,
            "component_version": 1,
            "description": "Local Mistral-7B model client for instruction-based generation (Ollama, LMStudio).",
            "label": "gemini-2.5-flash",
            "config": {
              "model": "gemini-2.5-flash",
              "api_key": "",
              "model_info": {
                "vision": true,
                "function_calling": true,
                "json_output": false,
                "structured_output": true,
                "family": "gemini"
              }
            }
          },
          "tools": [
            {
              "provider": "autogen_core.tools.FunctionTool",
              "component_type": "tool",
              "version": 1,
              "component_version": 1,
              "description": "A tool that performs basic arithmetic operations (addition, subtraction, multiplication, division).",
              "label": "Calculator Tool",
              "config": {
                "source_code": "def calculator(a: float, b: float, operator: str) -> str:\n    \"\"\"\n        A simple calculator function that performs basic arithmetic operations.\n        args:\n            a (float): The first number.\n            b (float): The second number.\n            operator (str): The arithmetic operator (+, -, *, /).\n        returns:\n            str: The result of the arithmetic operation as a string.\n    \"\"\"\n    try:\n        if operator == \"+\":\n            return str(a + b)\n        elif operator == \"-\":\n            return str(a - b)\n        elif operator == \"*\":\n            return str(a * b)\n        elif operator == \"/\":\n            if b == 0:\n                return \"Error: Division by zero\"\n            return str(a / b)\n        else:\n            return \"Error: Invalid operator. Please use +, -, *, or /\"\n    except Exception as e:\n        return f\"Error: {str(e)}\"\n",
                "name": "calculator",
                "description": "A simple calculator that performs basic arithmetic operations",
                "global_imports": [],
                "has_cancellation_support": false
              }
            }
          ],
          "model_context": {
            "provider": "autogen_core.model_context.UnboundedChatCompletionContext",
            "component_type": "chat_completion_context",
            "version": 1,
            "component_version": 1,
            "description": "An unbounded chat completion context that keeps a view of the all the messages.",
            "label": "UnboundedChatCompletionContext",
            "config": {}
          },
          "description": "An agent that provides assistance with ability to use tools.",
          "system_message": "You are a helpful assistant. Solve tasks carefully. When done, say TERMINATE.",
          "model_client_stream": true,
          "reflect_on_tool_use": true,
          "tool_call_summary_format": "{result}"
        }
      },
      {
        "provider": "autogen_agentchat.agents.AssistantAgent",
        "component_type": "agent",
        "version": 1,
        "component_version": 1,
        "description": "You are able to convert a math expression into plan english text. e.g.\ninput: 3+4=7 \noutput: three plus four equal to seven. ",
        "label": "SummarizerAgent",
        "config": {
          "name": "SummarizerAgent",
          "model_client": {
            "provider": "autogen_ext.models.openai.OpenAIChatCompletionClient",
            "component_type": "model",
            "version": 1,
            "component_version": 1,
            "description": "OpenAI GPT-4o-mini",
            "label": "OpenAI GPT-4o Mini",
            "config": {
              "model": "deepseek/deepseek-r1-0528:free",
              "base_url": "https://openrouter.ai/api/v1",
              "api_key": " ",
              "model_info": {
                "family": "deepseek",
                "structured_output": false,
                "vision": true,
                "function_calling": true,
                "json_output": false
              }
            }
          },
          "tools": [],
          "model_context": {
            "provider": "autogen_core.model_context.UnboundedChatCompletionContext",
            "component_type": "chat_completion_context",
            "version": 1,
            "component_version": 1,
            "description": "An unbounded chat completion context that keeps a view of the all the messages.",
            "label": "UnboundedChatCompletionContext",
            "config": {}
          },
          "description": "An agent that provides assistance with ability to use tools.",
          "system_message": "You are able to convert a math expression into plan english text. e.g.\ninput: 3+4=7 \noutput: three plus four equal to seven. \n\nWhen it comes to convertion to text, you are the last stage. return with the text \"TERMINATE\"",
          "model_client_stream": true,
          "reflect_on_tool_use": true,
          "tool_call_summary_format": "{result}"
        }
      }
    ],
    "termination_condition": {
      "provider": "autogen_agentchat.base.OrTerminationCondition",
      "component_type": "termination",
      "version": 1,
      "component_version": 1,
      "label": "OrTerminationCondition",
      "config": {
        "conditions": [
          {
            "provider": "autogen_agentchat.conditions.TextMentionTermination",
            "component_type": "termination",
            "version": 1,
            "component_version": 1,
            "description": "Terminate the conversation if a specific text is mentioned.",
            "label": "TextMentionTermination",
            "config": {
              "text": "TERMINATE"
            }
          },
          {
            "provider": "autogen_agentchat.conditions.MaxMessageTermination",
            "component_type": "termination",
            "version": 1,
            "component_version": 1,
            "description": "Terminate the conversation after a maximum number of messages have been exchanged.",
            "label": "MaxMessageTermination",
            "config": {
              "max_messages": 3,
              "include_agent_event": false
            }
          }
        ]
      }
    }
  }
}
```

In [4]:
termination = "Thanks for taking interview with us You may now leave."
candidate = AssistantAgent(
    name="InterviewCandidateAgent",
    description="A candidate for a job interview.",
    system_message="You are a candidate for a job interview. Answer questions concisely and clearly. Keep answer 1 line.",
    model_client=deepseek_client
)
interviewer = AssistantAgent(
    name="InterviewPanelAgent",
    description="An interviewer for a job interview.",
    system_message="You are an interviewer, name 'Googler', you are going to do a job interview. Ask relevant questions to assess the candidate's suitability for the role. When you are done with your interview pass to the HR Agent to conclude the interview. Keep question 1 line.",
    model_client=gemini_client
)
hr = AssistantAgent(
    name="HRAgent",
    description="An HR representative for a job interview.",
    system_message=f"You are an HR representative for a job interview. Ensure the interview process is smooth and professional. You only involve yourself during the begining and end of the interview. when interview is done say finally say the terminating statement - '{termination}'",
    model_client=deepseek_client
)
termination_text = TextMentionTermination(text=termination)

team = SelectorGroupChat(
    participants=[interviewer, candidate, hr],
    model_client=gemini_client,
    termination_condition=termination_text,
    max_turns=5
)

await Console(team.run_stream(task="Conduct a job interview of Greg for the position of Software Engineer. Greg reaches out to the HR to be connected with the panel. Interview panel will only ask 2 technical questions on microservice"))

---------- TextMessage (user) ----------
Conduct a job interview of Greg for the position of Software Engineer. Greg reaches out to the HR to be connected with the panel. Interview panel will only ask 2 technical questions on microservice
---------- TextMessage (HRAgent) ----------
### Step 1: HR Introduction  
**HR**: "Hello Greg, welcome to your interview for the Software Engineer position. We appreciate your time today. I'll now connect you with our technical interview panel, who will ask two questions focused on microservices. Best of luck!"  

*(Handing over to the interview panel)*  

### Step 2: Interview Panel Questions  
*(Simulating the panel — Greg, answer these technical questions concisely.)*  

1. **Question 1**: "Can you explain how *event-driven architecture* enhances decoupling in microservices, and describe a scenario where you might use Kafka or RabbitMQ to achieve this?"  

2. **Question 2**: "In a microservices environment, how would you handle *inter-service commu

TaskResult(messages=[TextMessage(id='fa952941-7816-424e-93a5-5930a813a44f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 8, 12, 26, 47, 157592, tzinfo=datetime.timezone.utc), content='Conduct a job interview of Greg for the position of Software Engineer. Greg reaches out to the HR to be connected with the panel. Interview panel will only ask 2 technical questions on microservice', type='TextMessage'), TextMessage(id='414a838e-ab68-4e27-bd66-0a697e4f3ee5', source='HRAgent', models_usage=RequestUsage(prompt_tokens=101, completion_tokens=1452), metadata={}, created_at=datetime.datetime(2026, 1, 8, 12, 27, 25, 141344, tzinfo=datetime.timezone.utc), content='### Step 1: HR Introduction  \n**HR**: "Hello Greg, welcome to your interview for the Software Engineer position. We appreciate your time today. I\'ll now connect you with our technical interview panel, who will ask two questions focused on microservices. Best of luck!"  \n\n*(Handing over to the 

In [46]:
from autogen_agentchat.conditions import TimeoutTermination
termination_text = "Task Completed"

timeout = TimeoutTermination(timeout_seconds=3000)
text_mention_terminator = TextMentionTermination(text=termination_text)
docker_executor = DockerCommandLineCodeExecutor(
    image="python:3.9-slim",
    work_dir="/tmp",
    extra_volumes={
        "/Users/achuth.iyyatil/Code/personal/stunning-spork": {
            "bind": "/tmp",
            "mode": "rw"
        }
    }
)

async with docker_executor:
    graph_agent = CodeExecutorAgent(
        name="CodeExecutorAssistantAgent",
        code_executor=docker_executor,
        model_client=gemini_client,
        description="An agent that creates mathematical equations and draws their graphs using matplotlib library."
    )
    assistant = AssistantAgent(
        name="AssistantAgent",
        model_client=gemini_client,
        system_message="You are a math test teacher who is going to give an equation to draw a graph of"
    )
    team = SelectorGroupChat(
        participants=[graph_agent, assistant],
        model_client=gemini_client,
        termination_condition=timeout | text_mention_terminator
    )
    await Console(team.run_stream(task="""
        1. Create a mathematical equation whose graph is a parabola
        2. Draw the graph using matplotlib and save it as 'parabola.png'
        3. Use docker cp command to copy the 'parabola.png' from container to host
        Execute all commands.
    """))


/var/folders/50/kmf0fqmd2sv_hvt0nxtxgt_00000gp/T/ipykernel_15094/1874178844.py:18: UserWarning: No approval function set for CodeExecutorAgent. This means code will be executed automatically without human oversight. For security, consider setting an approval_func to review and approve code before execution. See the CodeExecutorAgent documentation for examples of approval functions.
  graph_agent = CodeExecutorAgent(


---------- TextMessage (user) ----------

        1. Create a mathematical equation whose graph is a parabola
        2. Draw the graph using matplotlib and save it as 'parabola.png'
        3. Use docker cp command to copy the 'parabola.png' from container to host
        Execute all commands.
    
---------- CodeGenerationEvent (CodeExecutorAssistantAgent) ----------
Here's how to create a parabola graph using Matplotlib inside a Docker container and then copy the resulting image to your host system.

First, let's define the mathematical equation and the Python script to plot it.

### 1. Mathematical Equation for a Parabola

The most basic equation for a parabola opening upwards or downwards is:
$y = ax^2 + bx + c$

For simplicity, we will use:
$y = x^2$

### 2. Python Script (`plot_parabola.py`)

This script will generate the `x` and `y` values, plot them using Matplotlib, and save the graph as `parabola.png`.

```python
import matplotlib.pyplot as plt
import numpy as np
import os



ERROR:autogen_core:Error processing publish message for CodeExecutorAssistantAgent_aaad716f-7e89-42c5-bba2-ea242e81f4c9/aaad716f-7e89-42c5-bba2-ea242e81f4c9
Traceback (most recent call last):
  File "/Users/achuth.iyyatil/Code/personal/stunning-spork/.venv/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/achuth.iyyatil/Code/personal/stunning-spork/.venv/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/achuth.iyyatil/Code/personal/stunning-spork/.venv/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/achut

RuntimeError: RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_paid_tier_input_token_count, limit: 1000000, model: gemini-2.5-flash\nPlease retry in 47.311098867s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_paid_tier_input_token_count', 'quotaId': 'GenerateContentPaidTierInputTokensPerModelPerMinute', 'quotaDimensions': {'location': 'g